In [2]:
import os
import json
from sentence_transformers import SentenceTransformer, util

# Initialize a pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# List of category descriptions
category_descriptions = {
    "sorting": "sorting elements, bubble sort, quick sort, merge sort, heap sort",
    "searching": "searching elements, binary search, linear search, search algorithms",
    "mathematical operations": "mathematical operations, addition, subtraction, trigonometry, calculus",
    "data structures": "data structures, linked list, stack, queue, tree, graph",
    "string manipulation": "manipulating strings, string operations, string matching, regular expressions",
    "file handling": "file handling, reading files, writing files, file operations",
    "networking": "networking, socket programming, network protocols, client-server communication",
    "web scraping": "web scraping, parsing HTML, extracting data from websites, web crawling",
    "machine learning": "machine learning, deep learning, neural networks, training models, prediction",
    "other": "miscellaneous functions"
}

# Encode category descriptions
category_embeddings = {category: model.encode(description, convert_to_tensor=True) for category, description in category_descriptions.items()}

cache_file = 'func_name_to_category_cache.json'

# Load cached mappings if available
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        func_name_to_category = json.load(f)
else:
    func_name_to_category = {}

def map_func_name_to_category(func_name):
    if func_name in func_name_to_category:
        return func_name_to_category[func_name]
    
    func_name_embedding = model.encode(func_name, convert_to_tensor=True)
    similarities = {category: util.pytorch_cos_sim(func_name_embedding, embedding).item() for category, embedding in category_embeddings.items()}
    category = max(similarities, key=similarities.get)
    func_name_to_category[func_name] = category

    # Save the updated mappings to the cache file
    with open(cache_file, 'w') as f:
        json.dump(func_name_to_category, f)
    
    return category

# Test the mapping function with some example function names
func_names = ["sort_array", "binary_search", "calculate_sum", "linked_list_operations", "show_prediction_labels_on_image"]

for func_name in func_names:
    category = map_func_name_to_category(func_name)
    print(f"Function Name: {func_name} -> Category: {category}")


C:\Users\barte\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Function Name: sort_array -> Category: sorting
Function Name: binary_search -> Category: searching
Function Name: calculate_sum -> Category: mathematical operations
Function Name: linked_list_operations -> Category: data structures
Function Name: show_prediction_labels_on_image -> Category: machine learning


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Define the mapping between function names and categories
label_mapping = {
    "sorting": 0,
    "searching": 1,
    "mathematical operations": 2,
    "data structures": 3,
    "string manipulation": 4,
    "file handling": 5,
    "networking": 6,
    "web scraping": 7,
    "machine learning": 8,
    "other": 9
}

# Load the CodeSearchNet dataset
dataset = load_dataset("code_search_net", "python")

# Pre-process the dataset
def preprocess_function(examples):
    examples["label"] = [label_mapping.get(map_func_name_to_category(func), label_mapping["other"]) for func in examples["func_name"]]
    return tokenizer(examples["whole_func_string"], truncation=True, padding="max_length", max_length=128)

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# Select a shard of the dataset for quicker processing
shard_size = 16000  # Define the shard size
train_dataset = dataset['train'].shard(index=0, num_shards=len(dataset["train"]) / shard_size)
validation_dataset = dataset['test'].shard(index=0, num_shards=len(dataset["train"]) / shard_size)

# Tokenize the datasets
tokenized_train_datasets = train_dataset.map(preprocess_function, batched=True, batch_size=1000)
tokenized_validation_datasets = validation_dataset.map(preprocess_function, batched=True, batch_size=1000)

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=len(label_mapping))

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=100,
    weight_decay=0.01,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_validation_datasets,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
stats = trainer.evaluate()
print(f"Stats of the trained model: {stats}")

# Save the model
model.save_pretrained("p1Model")
tokenizer.save_pretrained("p1Tokenizer")

C:\Users\barte\AppData\Roaming\Python\Python310\site-packages\datasets\load.py:1491: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Map: 100%|██████████| 861/861 [00:00<00:00, 5280.92 examples/s]
c:\Users\barte\miniconda3\envs\py310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Rob

{'loss': 2.0916, 'grad_norm': 5.750487327575684, 'learning_rate': 1.9e-05, 'epoch': 0.1}


 10%|█         | 200/2000 [00:53<08:27,  3.55it/s]

{'loss': 2.0054, 'grad_norm': 5.591728210449219, 'learning_rate': 1.8e-05, 'epoch': 0.2}


 15%|█▌        | 300/2000 [01:23<07:44,  3.66it/s]

...

                                                   
100%|██████████| 2000/2000 [24:35<00:00,  1.36it/s]


{'eval_loss': 1.555184006690979, 'eval_runtime': 8.3436, 'eval_samples_per_second': 103.193, 'eval_steps_per_second': 6.472, 'epoch': 2.0}
{'train_runtime': 1475.6221, 'train_samples_per_second': 21.686, 'train_steps_per_second': 1.355, 'train_loss': 1.5669732208251954, 'epoch': 2.0}


100%|██████████| 54/54 [00:08<00:00,  6.70it/s]


Evaluation results: {'eval_loss': 1.555184006690979, 'eval_runtime': 8.0904, 'eval_samples_per_second': 106.423, 'eval_steps_per_second': 6.675, 'epoch': 2.0}


('p1_saved_tokenizer\\tokenizer_config.json',
 'p1_saved_tokenizer\\special_tokens_map.json',
 'p1_saved_tokenizer\\vocab.json',
 'p1_saved_tokenizer\\merges.txt',
 'p1_saved_tokenizer\\added_tokens.json',
 'p1_saved_tokenizer\\tokenizer.json')

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Define the label mapping
labels = {
    0: "sorting",
    1: "searching",
    2: "mathematical operations",
    3: "data structures",
    4: "string manipulation",
    5: "file handling",
    6: "networking",
    7: "web scraping",
    8: "machine learning",
    9: "other"
}

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("p1Model")
tokenizer = AutoTokenizer.from_pretrained("p1Tokenizer")

# Create a pipeline for text classification
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Define the code snippets
code_snippets = [
    "sorted(l, key=lambda x: (-int(x[1]), x[0]))",
    "def multiply(a, b): return a * b",
    "def divide(a, b): return a / b",
    "words = text.split(' ') \\ last = words[0] \\ for word in words: \\ if word > last: \\ last = word \\ return last",
    "def loss_fn(y_true, y_pred): return tf.reduce_mean(tf.square(y_true - y_pred))",
    "def func(x, y): return x * y + x"
]

# Classify the code snippets
for code_snippet in code_snippets:
    # Classify the code snippet
    classification_result = classifier(code_snippet)

    # Extract the label and map it to the class name
    label = int(classification_result[0]['label'].split('_')[-1])  # Extract numerical label from 'LABEL_0', 'LABEL_1', etc.
    class_name = labels.get(label, "Unknown")

    print(f"Code Snippet: {code_snippet}")
    print(f"Classification: {class_name}, Score: {classification_result[0]['score']}\n")


Code Snippet: sorted(l, key=lambda x: (-int(x[1]), x[0]))
Classification: sorting, Score: 0.7280794382095337

Code Snippet: def multiply(a, b): return a * b
Classification: mathematical operations, Score: 0.3808503746986389

Code Snippet: def divide(a, b): return a / b
Classification: mathematical operations, Score: 0.3947147727012634

Code Snippet: words = text.split(' ') \ last = words[0] \ for word in words: \ if word > last: \ last = word \ return last
Classification: string manipulation, Score: 0.5963256359100342

Code Snippet: def loss_fn(y_true, y_pred): return tf.reduce_mean(tf.square(y_true - y_pred))
Classification: other, Score: 0.2912917733192444

Code Snippet: def func(x, y): return x * y + x
Classification: other, Score: 0.5813041925430298

